In [4]:
%matplotlib inline
import numpy as np
import numba
from time import time
from itertools import izip

# Universal Functions

In [34]:
# Some common universal functions: +, -, /, *
a = np.array([1., 2., 3.])
b = np.array([4., 5., 6.])

# The operation is made element-wise
print a + b
print a - b
print a * b
print a / b

[ 5.  7.  9.]
[-3. -3. -3.]
[  4.  10.  18.]
[ 0.25  0.4   0.5 ]


In [35]:
# Broadcast rules apply
print np.array([1, 2, 3, 4]) + 5

[6 7 8 9]


##Creating our own ufunc

In [36]:
def my_func(x):
    if x > 10:
        return 0.0 # NOTE: This function is weird, as it returns different data types depending on the input.
    return x + 1

data = np.array([1,2,30,40])
# This wont work
try:
    my_func(data)
except Exception:
    print "I can\'t do it :("

I can't do it :(


### frompyfunc

In [37]:
def my_func(x):
    if x > 10:
        return 0.0 # NOTE: This function is weird, as it returns different data types depending on the input.
    return x + 1

data = np.array([1,2,30,40])

f = np.frompyfunc(my_func, 1, 1)
print f(data)
# Note: frompyfunc will always return a numpy array of python objects
print f(data).dtype

[2 3 0.0 0.0]
object


###vectorize

In [38]:
# From the docs (https://github.com/numpy/numpy/blob/v1.9.1/numpy/lib/function_base.py#L1627)
# The `vectorize` function is provided primarily for convenience, not for
# performance. The implementation is essentially a for loop.
g = np.vectorize(my_func)
print g(data)

[2 3 0 0]


In [40]:
# Just to quick explain (Again?) the weird vectorize output type
# It is really simple: If no output type is given, vectorize will run once with the first array value,
# so that he can decide the data type. Example:

def raw_float_func(x):
    print ".",
    return 1.0
float_ufunc = np.vectorize(raw_float_func)

def raw_int_func(x):
    print ".",
    return 1
int_ufunc = np.vectorize(raw_int_func)

data = np.array(['a', 'b', 'c'])
result = float_ufunc(data)
print result, result.dtype

result = int_ufunc(data)
print result, result.dtype

# We can also choose the output type when vectorizing
int_to_float_ufunc = np.vectorize(raw_int_func, otypes=[np.int])
result = int_to_float_ufunc(data)
print result, result.dtype

. . . . [ 1.  1.  1.] float64
. . . . [1 1 1] int32
. . . [1 1 1] int32


##Some timing

In [5]:
# Which one is faster?
def my_func(x):
    if x > 10:
        return 0.0
    return x + 1

f = np.frompyfunc(my_func, 1, 1)
g = np.vectorize(my_func)

def raw_python(data):
    o2 = []
    for i in data:
        o2.append(my_func(i))

def raw_numpy(data):
    mask = data > 10
    data += 1
    data[mask] = 0.0


data = np.arange(int(1e+7))
raw_data = xrange(int(1e+7))
                 
%timeit raw_python(raw_data)
%timeit f(data)
%timeit g(data)
%timeit raw_numpy(data)

1 loops, best of 3: 2.27 s per loop
1 loops, best of 3: 1.42 s per loop
1 loops, best of 3: 2.52 s per loop
10 loops, best of 3: 23.2 ms per loop


## Advantages and a bit more about ufuncs

In [47]:
# And what are the advantages, since it seems slow??
# We can vectorize python functions
# Example:

print chr(42)

# Exception:
# print chr(np.array([1,2,3,4,5]))

ufunc_chr = np.vectorize(chr)
ufunc_chr(np.array([42,43,44,45,46]))

*


array(['*', '+', ',', '-', '.'], 
      dtype='|S1')

In [51]:
# BOOKMEETING TESTS ----------------------------------------------
import math
ufunc_sin = np.vectorize(math.sin)
frompyfunc = np.frompyfunc(math.sin, 1, 1)

%timeit frompyfunc(data)
%timeit ufunc_sin(data)
%timeit np.sin(data)
# ----------------------------------------------------------------

1 loops, best of 3: 1.01 s per loop
1 loops, best of 3: 1.33 s per loop
10 loops, best of 3: 74.2 ms per loop


In [7]:
# Example of user-defined ufunc with multiple inputs
def my_ufunc(x, y):
    if x > y:
        return x
    else:
        return x + y

f = np.vectorize(my_ufunc, otypes=[np.float64])
print f(np.array([1,2,3]), np.array([0,5,0]))

# And broadcasting works on those cases!
print f(np.array([1,2,3]), 5)

[ 1.  7.  3.]
[ 6.  7.  8.]


In [57]:
# Example of user-defined ufunc with multiple outputs
def my_ufunc(x):
    return x, x + 1

f = np.vectorize(my_ufunc, otypes=[np.float64, np.float64])
f(np.array([1,2,3]))

(array([ 1.,  2.,  3.]), array([ 2.,  3.,  4.]))

# Using numba to speed things up

"Numba gives you the power to speed up your applications with high performance functions written directly in Python."

http://numba.pydata.org/numba-doc/0.20.0/user/overview.html

In [58]:
def f(x, y):
    result = 0.0
    for i in xrange(x):
        result += i ** 2
    for j in xrange(y):
        result += j ** 2
    return result

def py_f(x_vec, y_vec):
    results = []
    for i, j in izip(x_vec, y_vec):
        results.append(f(i, j))
    return results

%timeit py_f(xrange(5000), [5] * 5000)

1 loops, best of 3: 1.24 s per loop


In [59]:
def f(x, y):
    result = 0.0
    for i in xrange(x):
        result += i ** 2
    for j in xrange(y):
        result += j ** 2
    return result
ufunc_f = np.vectorize(f, otypes=[np.float64])

%timeit ufunc_f(np.arange(5000), 5)

1 loops, best of 3: 1.19 s per loop


In [65]:
@numba.jit(nopython=True)
def f(x, y):
    result = 0.0
    for i in xrange(x):
        result += i ** 2
    for j in xrange(y):
        result += j ** 2
    return result
numba_ufunc_f = np.vectorize(f, otypes=[np.float64])

# option (nopython=True)
# From the docs:
# A Numba compilation mode that generates code that does not access the Python C API.
# This compilation mode produces the highest performance code.
# http://numba.pydata.org/numba-doc/0.20.0/glossary.html#term-nopython-mode
numba_ufunc_f(np.arange(5000), 5)

%timeit numba_ufunc_f(np.arange(5000), 5)
%timeit numba_ufunc_f(np.arange(5000, dtype=np.float64), 5)

100 loops, best of 3: 17.6 ms per loop
The slowest run took 4.12 times longer than the fastest. This could mean that an intermediate result is being cached 
100 loops, best of 3: 17.7 ms per loop


In [8]:
# BOOKMEETING TESTS ----------------------------------------------
@numba.jit(nopython=True)
def f(x):
    if x > 10:
        return float(0)
    return 1

print f(1.0)
print f(11.0)
# ----------------------------------------------------------------

1.0
0.0


In [89]:
@numba.vectorize(nopython=True)
def f(x, y):
    result = 0.0
    for i in xrange(x):
        result += i ** 2
    for j in xrange(y):
        result += j ** 2
    return result

%timeit f(np.arange(5000), 5)

The slowest run took 5.81 times longer than the fastest. This could mean that an intermediate result is being cached 
100 loops, best of 3: 17.4 ms per loop


In [103]:
@numba.vectorize(nopython=True)
def mysin(x):
    return np.sin(x)

@numba.jit(nopython=True)
def my_slow_sin(x):
    return math.sin(x)
vec_my_slow_sin = np.vectorize(my_slow_sin)


@numba.jit(nopython=True)
def my_np_sin(x):
    return np.sin(x)

vec_np_sin = np.vectorize(my_np_sin)
vec_my_slow_sin(data)

%timeit mysin(data)
%timeit vec_my_slow_sin(data)
%timeit my_np_sin(data)

10 loops, best of 3: 70.6 ms per loop
1 loops, best of 3: 2.08 s per loop
10 loops, best of 3: 80.4 ms per loop


In [122]:
# BOOKMEETING TESTS ----------------------------------------------
print my_slow_sin.inspect_llvm().values()[0]
# ----------------------------------------------------------------

; ModuleID = '<string>'
target datalayout = "e-m:e-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-pc-windows-msvc"

@PyExc_RuntimeError = external global i8
@.const.my_slow_sin = internal constant [12 x i8] c"my_slow_sin\00"
@".const.Fatal_error:_missing__dynfunc.Closure" = internal constant [38 x i8] c"Fatal error: missing _dynfunc.Closure\00"
@.const.missing_Environment = internal constant [20 x i8] c"missing Environment\00"

; Function Attrs: nounwind
define i32 @"__main__.my_slow_sin$51.float64"(double* nocapture %retptr, { i8*, i32 }** nocapture readnone %excinfo, i8* nocapture readnone %env, double %arg.x) #0 {
entry:
  %.16 = tail call double @llvm.sin.f64(double %arg.x)
  store double %.16, double* %retptr, align 8
  ret i32 0
}

; Function Attrs: nounwind readnone
declare double @llvm.sin.f64(double) #1

define i8* @"wrapper.__main__.my_slow_sin$51.float64"(i8* %py_closure, i8* %py_args, i8* nocapture readnone %py_kws) {
entry:
  %.5 = alloca i8*, align 8
  %.6 = cal

In [150]:
# What else can we do?

# Option nogil=True:
# Code running with the GIL (Global Interpreter Lock) released runs concurrently with other threads
# executing Python or Numba code

# Inspect the numba function with inspect_types, inspect_llvm and inspect_asm

# Play around with numbaPro -> Allows code generation for CPU, Multicore CPU and GPU

# Ideas for next meetings?
# Overall discussion?
